# Exercises (with solutions)


In [ ]:
import numpy as np

from astropy import table
from astropy.table import Table

%matplotlib inline
from matplotlib import pyplot as plt

### Read the data

To start with, read in the two data files representing the master source list and observations source list.  The fields for the two tables are respectively documented in:

- [master_sources](http://cxc.harvard.edu/csc/columns/master.html)
- [obs_sources](http://cxc.harvard.edu/csc/columns/persrc.html)

In [ ]:
master_sources = Table.read('cdfs_master_sources.fits')
obs_sources = Table.read('cdfs_obs_sources.fits')

**`master_sources`**

Each distinct X-ray source identified on the sky is represented in the catalog by a single "master source" entry and one or more "source observation" entries, one for each observation in which the source has been detected. The master source entry records the best estimates of the properties of a source, based on the data extracted from the set of observations in which the source has been detected.  The subset of fields in our exercise table file are:

Name | Description
------ | ------------
msid  | Master source ID
name  | Source name in the Chandra catalog
ra  | Source RA (deg)
dec | Source Dec (deg)

**`obs_sources`**

The individual source entries record all of the properties about a detection extracted from a single observation, as well as associated file-based data products, which are observation-specific.  The subset of fields in our exercise table file are:

Name | Description
------ | ------------
obsid | Observation ID
obi | Observation interval
targname | Target name
gti_obs | Observation date
flux_aper_b | Broad band (0.5 - 7 keV) flux (erg/cm2/sec)
src_cnts_aper_b | Broad band source counts
ra_b | Source RA (deg)
dec_b | Source Dec (deg)
livetime | Observation duration (sec)
posid | Position ID
theta | Off-axis angle (arcmin)
msid | Master source ID

### Exploring the data
Do the following to explore the two tables:

- Display the data for each table in IPython notebook using the normal way of showing the value of a variable.
- Get a list of the column names for each table.  *Hint*: use `<TAB>` completion to conveniently discover all the attributes and methods (e.g., type `master_sources.` and then hit the `<TAB>` key).
- Find the length of each table.
- Find the column datatypes for each table.

Normally you display a table in IPython notebook by entering the variable name in a cell and pressing `shift-Enter`.  In a terminal session the default method is using something like `print(my_table)`.  In both cases the `Table` object prefers to display only a screenful of data to prevent having a zillion lines of output if the table is huge.  If you really want to see all the data you can use the [Table.pprint](http://astropy.readthedocs.org/en/stable/api/astropy.table.Table.html#astropy.table.Table.pprint) method. If you are using a Jupyter notebook interface, try the `show_in_notebook()` method.

- Display all the rows of the `master_sources` table using its `pprint()` method.
- If you are working in a regular terminal window (not IPython notebook), try the `more()` method as well.

In [ ]:
master_sources.pprint()

In [ ]:
obs_sources.show_in_notebook()

### Modifying tables
For our analysis we don't actually need the `obi` (observation interval) column in the `obs_sources` table.

- Remove the `obi` column from the `obs_sources` table.

The `gti_obs` column name is a bit obscure (GTI is a good time interval, FWIW).

- Rename the `gti_obs` column to `obs_date`.

It would be nice to have a count rate in addition to the source counts.

- Add a new column `src_rate_aper_b` which is the source counts divided by observation duration in sec.

Some of the sources have a negative net flux in the broad band.

In [ ]:
obs_sources.remove_column('obi')

In [ ]:
obs_sources.rename_column("gti_obs", "obs_date")

In [ ]:
obs_sources['src_rate_aper_b'] = obs_sources['src_cnts_aper_b'] / obs_sources['livetime']

### Looking at the observation source data
For each source detected in an individual observation (in the `obs_sources` table), let's look at the source flux values.

- Use the matplotlib [`hist()`]( http://matplotlib.org/api/pyplot_api.html?highlight=pyplot.hist#matplotlib.pyplot.hist) function to make a histogram of the source fluxes.  Since the fluxes vary by orders of magnitude,
  use the `numpy.log10` to put the fluxes in log space.

- Also make the same plot but using only sources within 4 arcmin of the center.  *HINT*: use a boolean mask to select values of `theta` that are less than 4.0.

In [ ]:
plt.figure()
plt.hist(np.log10(obs_sources['flux_aper_b']))
plt.show()

In [ ]:
mask = obs_sources['theta'] < 4.0
plt.figure()
plt.hist(np.log10(obs_sources[mask]['flux_aper_b']))
plt.show()

### Join the master_sources and obs_sources tables

The `master_sources` and `obs_sources` tables share a common `msid` column. What we now want is to join the master RA and Dec positions and master source names with the individual observations table.

- Use the [table.join()](http://astropy.readthedocs.org/en/stable/table/operations.html#join) function to make a single table called `sources` that has the master RA, Dec, and name included for each observation source.

*HINT*: the defaults for `keys` and `join_type='inner'`  are correct in this case, so the simplest possible call to `join()` will work!

- *Intermediate*: Is the length of the new `sources` the same as `obs_sources`?  What happened?

- *Advanced*: Make a scatter plot of the RA (x-axis) and Dec (y-axis) difference between the master source position and the observation source position.  You'll need to use `coordinates`!

In [ ]:
sources = table.join(master_sources, obs_sources, join_type='inner')
len(sources), len(master_sources), len(obs_sources)

In [ ]:
sources.colnames

In [ ]:
from astropy.coordinates import SkyCoord
import astropy.units as u

src_coord = SkyCoord(ra=sources['ra'], dec=sources['dec'], unit=(u.hourangle, u.deg))
obs_coord = SkyCoord(ra=sources['ra_b'], dec=sources['dec_b'], unit=(u.hourangle, u.deg))
d_ra = src_coord.ra - obs_coord.ra
d_dec = src_coord.dec - obs_coord.dec

# convert degrees to arcsec
plt.scatter(d_ra.arcsec, d_dec.arcsec)

### Grouped properties of `sources`

Finally, we can look at the variability properties of sources in the CDFS using the [`group_by()`](http://astropy.readthedocs.org/en/stable/table/operations.html#id2) functionality.  

This method makes a new table in which all of the sources with identical master ID are next to each other.

- Make a new table `g_sources` which is the `sources` table grouped by the `msid` key using the `group_by()` method.

The `g_sources` table is just a regular table with all of the `sources` in a particular order.  The attribute `g_sources.groups` is an object that provides access to the `msid` subgroups.  You can access the $i^{th}$ group with `g_sources.groups[i]`.

In addition, the `g_sources.groups.indices` attribute is an array with the indicies of the group boundaries.

- Using `np.diff()` find the number of repeat observations of each master sources.  *HINT*: use the indices, Luke.
- Print the 50th group and note which columns are the same for all group members and which are different.  Does this make sense?  In these few observations how many different target names were provided by observers?

In [ ]:
g_sources = sources.group_by('msid')

In [ ]:
np.diff(g_sources.groups.indices)

In [ ]:
g_sources.groups[50]

### Aggregation

The real power of grouping comes in the ability to create aggregate values for each of the groups, for instance the mean flux for each unique source.  This is done with the [`aggregate()`](http://astropy.readthedocs.org/en/stable/table/operations.html#aggregation) method, which takes a function reference as its input.  This function must take as input an array of values and return a single value.

`aggregate` returns a new table that has a length equal to the number of groups.

- Compute the mean of all columns for each unique source (i.e. each group) using `aggregate` and the `np.mean` function.  Call this table `g_sources_mean`.
- Notice that aggregation cannot form a mean for certain columns and these are dropped from the output.  Use the `join()` function to restore the `master_sources` information to `g_sources_mean`.

In [ ]:
g_sources_mean = table.join(g_sources.groups.aggregate(np.mean), master_sources, keys=['msid'], join_type='inner')
g_sources_mean

[Back to top](#Tables-introduction)